In [1]:
import pulp as plp
from pulp import LpProblem, LpMaximize, LpVariable, lpSum
from domain.entities import DayOfTheWeek, Employee, Patient
import itertools

### Employee Schedules
We limited the schedules from 7:00 to 12:00 for clarity
| Employee 1 | Monday | Tuesday | Wednesday | Thursday | Friday | Saturday | Sunday |
| --- | --- | --- | --- | --- | --- | --- | --- |
| 07:00 - 7:30  | G1 | G2 | -- | G1 | G3 | -- | -- |
| 07:30 - 8:00  | G1 | G2 | -- | G1 | G3 | -- | -- |
| 08:00 - 8:30  | G1 | G2 | -- | G1 | G3 | -- | -- |
| 08:30 - 9:00  | -- | -- | -- | -- | G3 | -- | -- |
| 09:00 - 9:30  | -- | -- | -- | -- | G3 | -- | -- |
| 09:30 - 10:00 | G2 | G3 | -- | G3 | -- | -- | -- |
| 10:00 - 10:30 | G2 | G3 | -- | G3 | -- | -- | -- |
| 10:30 - 11:00 | G2 | G3 | -- | G3 | -- | -- | -- |
| 11:00 - 11:30 | G1 | G3 | -- | G3 | -- | -- | -- |
| 11:30 - 12:00 | G1 | G3 | -- | G3 | -- | -- | -- |

| Employee 2 | Monday | Tuesday | Wednesday | Thursday | Friday | Saturday | Sunday |
| --- | --- | --- | --- | --- | --- | --- | --- |
| 07:00 - 7:30  | -- | -- | G1 | G2 | -- | -- | -- |
| 07:30 - 8:00  | -- | G1 | G3 | G2 | -- | -- | -- |
| 08:00 - 8:30  | -- | G1 | G3 | -- | -- | -- | -- |
| 08:30 - 9:00  | -- | -- | -- | -- | -- | -- | -- |
| 09:00 - 9:30  | -- | -- | -- | -- | -- | -- | -- |
| 09:30 - 10:00 | G3 | G1 | G2 | -- | -- | -- | -- |
| 10:00 - 10:30 | G3 | G1 | G2 | -- | -- | -- | -- |
| 10:30 - 11:00 | G3 | -- | G2 | G1 | -- | -- | -- |
| 11:00 - 11:30 | G3 | -- | G2 | G1 | -- | -- | -- |
| 11:30 - 12:00 | G3 | -- | G2 | G1 | -- | -- | -- |

| Employee 3 | Monday | Tuesday | Wednesday | Thursday | Friday | Saturday | Sunday |
| --- | --- | --- | --- | --- | --- | --- | --- |
| 07:00 - 7:30  | -- | G3 | G2 | -- | G2 | -- | -- |
| 07:30 - 8:00  | -- | G3 | G2 | -- | G2 | -- | -- |
| 08:00 - 8:30  | -- | -- | G2 | -- | G2 | -- | -- |
| 08:30 - 9:00  | -- | -- | -- | -- | -- | -- | -- |
| 09:00 - 9:30  | -- | G1 | -- | -- | G3 | -- | -- |
| 09:30 - 10:00 | -- | -- | G1 | -- | G1 | -- | -- |
| 10:00 - 10:30 | -- | G1 | G1 | -- | G1 | -- | -- |
| 10:30 - 11:00 | -- | G1 | G1 | -- | G1 | -- | -- |
| 11:00 - 11:30 | -- | -- | -- | -- | -- | -- | -- |
| 11:30 - 12:00 | -- | -- | -- | -- | -- | -- | -- |

| Occupation | Monday | Tuesday | Wednesday | Thursday | Friday | Saturday | Sunday |
| --- | --- | --- | --- | --- | --- | --- | --- |
| 07:00 - 7:30  | G1----- | G2----G3 | ---G1-G2 | G1-G2--- | G3----G2 | -- | -- |
| 07:30 - 8:00  | G1----- | G2-G1-G3 | ---G3-G2 | G1-G2--- | G3----G2 | -- | -- |
| 08:00 - 8:30  | G1----- | G2-G1--- | ---G3-G2 | G1------ | G3----G2 | -- | -- |
| 08:30 - 9:00  | ------- | -------- | -------- | -------- | G3------ | -- | -- |
| 09:00 - 9:30  | ------- | ------G1 | -------- | -------- | G3----G3 | -- | -- |
| 09:30 - 10:00 | G2-G3-- | G3-G1--- | ---G2-G1 | G3------ | ------G1 | -- | -- |
| 10:00 - 10:30 | G2-G3-- | G3-G1-G1 | ---G2-G1 | G3------ | ------G1 | -- | -- |
| 10:30 - 11:00 | G2-G3-- | G3----G1 | ---G2-G1 | G3-G1--- | ------G1 | -- | -- |
| 11:00 - 11:30 | G1-G3-- | G3------ | ---G2--- | G3-G1--- | -------- | -- | -- |
| 11:30 - 12:00 | G1-G3-- | G3------ | ---G2--- | G3-G1--- | -------- | -- | -- |





### Patient Schedules
We limited the schedules from 7:00 to 12:00 for clarity
| Patient 1 | Monday | Tuesday | Wednesday | Thursday | Friday | Saturday | Sunday |
| --- | --- | --- | --- | --- | --- | --- | --- |
| 07:00 - 7:30  | -- | -- | -- | -- | -- | -- | -- |
| 07:30 - 8:00  | -- | -- | -- | -- | G3 | -- | -- |
| 08:00 - 8:30  | -- | -- | -- | -- | G3 | -- | -- |
| 08:30 - 9:00  | -- | -- | -- | -- | G3 | -- | -- |
| 09:00 - 9:30  | -- | -- | -- | -- | G3 | -- | -- |
| 09:30 - 10:00 | G2 | -- | -- | -- | -- | -- | -- |
| 10:00 - 10:30 | G2 | -- | -- | -- | -- | -- | -- |
| 10:30 - 11:00 | -- | -- | -- | -- | -- | -- | -- |
| 11:00 - 11:30 | G1 | -- | -- | -- | -- | -- | -- |
| 11:30 - 12:00 | -- | -- | -- | -- | -- | -- | -- |

| Patient 2 | Monday | Tuesday | Wednesday | Thursday | Friday | Saturday | Sunday |
| --- | --- | --- | --- | --- | --- | --- | --- |
| 07:00 - 7:30  | -- | G3 | -- | -- | -- | -- | -- |
| 07:30 - 8:00  | -- | G3 | -- | -- | G3 | -- | -- |
| 08:00 - 8:30  | -- | -- | -- | -- | G3 | -- | -- |
| 08:30 - 9:00  | -- | -- | -- | -- | G3 | -- | -- |
| 09:00 - 9:30  | -- | -- | -- | -- | G3 | -- | -- |
| 09:30 - 10:00 | -- | -- | -- | -- | -- | -- | -- |
| 10:00 - 10:30 | -- | -- | -- | -- | -- | -- | -- |
| 10:30 - 11:00 | -- | -- | -- | -- | -- | -- | -- |
| 11:00 - 11:30 | -- | -- | -- | -- | -- | -- | -- |
| 11:30 - 12:00 | -- | -- | -- | -- | -- | -- | -- |

| Patient 3 | Monday | Tuesday | Wednesday | Thursday | Friday | Saturday | Sunday |
| --- | --- | --- | --- | --- | --- | --- | --- |
| 07:00 - 7:30  | -- | -- | G2 | -- | -- | -- | -- |
| 07:30 - 8:00  | -- | -- | G2 | -- | -- | -- | -- |
| 08:00 - 8:30  | -- | -- | G2 | -- | -- | -- | -- |
| 08:30 - 9:00  | -- | -- | -- | -- | -- | -- | -- |
| 09:00 - 9:30  | -- | -- | -- | -- | -- | -- | -- |
| 09:30 - 10:00 | -- | -- | G2 | -- | -- | -- | -- |
| 10:00 - 10:30 | -- | -- | G2 | G3 | -- | -- | -- |
| 10:30 - 11:00 | -- | -- | G2 | G3 | -- | -- | -- |
| 11:00 - 11:30 | -- | -- | -- | G3 | -- | -- | -- |
| 11:30 - 12:00 | -- | -- | -- | -- | -- | -- | -- |


In [2]:
# define dummy employees
employee1 = Employee(1)
employee2 = Employee(2)
employee3 = Employee(3)

employee1.schedule.add_items([
        (1, DayOfTheWeek.MONDAY, 7,8.5),
        (1, DayOfTheWeek.MONDAY, 9.5,12),
        (1, DayOfTheWeek.MONDAY, 12.5,14),
        (1, DayOfTheWeek.TUESDAY, 7,8.5),
        (1, DayOfTheWeek.TUESDAY, 9.5,12),
        (1, DayOfTheWeek.THURSDAY,7,8.5),
        (1, DayOfTheWeek.THURSDAY,9.5,12),
        (1, DayOfTheWeek.FRIDAY, 7,9.5),
    ])

employee2.schedule.add_items([
        (1, DayOfTheWeek.MONDAY, 9.5,12),
        (1, DayOfTheWeek.TUESDAY, 7.5,8.5),
        (1, DayOfTheWeek.TUESDAY, 9.5,10.5),
        (1, DayOfTheWeek.WEDNESDAY, 7,7.5),
        (1, DayOfTheWeek.WEDNESDAY,7.5,8.5),
        (1, DayOfTheWeek.WEDNESDAY,9.5,12),
        (1, DayOfTheWeek.THURSDAY, 7,8),
        (1, DayOfTheWeek.THURSDAY, 10.5,12)
    ])

employee3.schedule.add_items([
    (1, DayOfTheWeek.TUESDAY,7,8),
    (1, DayOfTheWeek.TUESDAY,9,9.5),
    (1, DayOfTheWeek.TUESDAY,10,11),
    (1, DayOfTheWeek.WEDNESDAY,7,8.5),
    (1, DayOfTheWeek.WEDNESDAY,9.5,11),
    (1, DayOfTheWeek.FRIDAY,7,8.5),
    (1, DayOfTheWeek.FRIDAY,9,9.5),
    (1, DayOfTheWeek.FRIDAY,9.5,11),
])

employees = [employee1, employee2, employee3]


In [3]:
# define dummy patients
patient1 = Patient(1)
patient2 = Patient(2)
patient3 = Patient(3)

patient1.schedule.add_items([
        (1, DayOfTheWeek.MONDAY, 9.5,10.5),
        (1, DayOfTheWeek.MONDAY, 11,11.5),
        (1, DayOfTheWeek.FRIDAY, 7.5,9.5),
    ])

patient2.schedule.add_items([
    (1, DayOfTheWeek.TUESDAY,7,8),
    (1, DayOfTheWeek.FRIDAY,7.5,9.5),
    ])

patient3.schedule.add_items([
    (1, DayOfTheWeek.WEDNESDAY,7,8.5),
    (1, DayOfTheWeek.WEDNESDAY,9.5,11),
    (1, DayOfTheWeek.THURSDAY,10,11.5),
])

patients = [patient1, patient2, patient3]

In [9]:
ct = list(itertools.combinations((employees), 2))
ct.extend(employees)
print(ct)

[(<domain.entities.Employee object at 0x0000024CBA1855D0>, <domain.entities.Employee object at 0x0000024CBA1856D0>), (<domain.entities.Employee object at 0x0000024CBA1855D0>, <domain.entities.Employee object at 0x0000024CBA185910>), (<domain.entities.Employee object at 0x0000024CBA1856D0>, <domain.entities.Employee object at 0x0000024CBA185910>), <domain.entities.Employee object at 0x0000024CBA1855D0>, <domain.entities.Employee object at 0x0000024CBA1856D0>, <domain.entities.Employee object at 0x0000024CBA185910>]


In [16]:
ct_check = []
for therapist_combo in ct:
    if type(therapist_combo) == tuple:
        if therapist_combo[0].schedule.get_numeric_template() == therapist_combo[1].schedule.get_numeric_template():
                if therapist_combo not in ct_check:
                    ct_check.append(therapist_combo)
    else:
        ct_check.append(therapist_combo)
print(ct_check)

[<domain.entities.Employee object at 0x0000024CBA1855D0>, <domain.entities.Employee object at 0x0000024CBA1856D0>, <domain.entities.Employee object at 0x0000024CBA185910>]


In [17]:
cp = list(itertools.combinations((patients), 8 ))
cp.extend(itertools.combinations((patients), 7 ))
cp.extend(itertools.combinations((patients), 6 ))
cp.extend(itertools.combinations((patients), 5 ))
cp.extend(itertools.combinations((patients), 4 ))
cp.extend(itertools.combinations((patients), 3 ))
cp.extend(itertools.combinations((patients), 2 ))
cp.extend(patients)
print(cp)

[(<domain.entities.Patient object at 0x0000024CBA15DFD0>, <domain.entities.Patient object at 0x0000024CBA15DBD0>, <domain.entities.Patient object at 0x0000024CBA15CA10>), (<domain.entities.Patient object at 0x0000024CBA15DFD0>, <domain.entities.Patient object at 0x0000024CBA15DBD0>), (<domain.entities.Patient object at 0x0000024CBA15DFD0>, <domain.entities.Patient object at 0x0000024CBA15CA10>), (<domain.entities.Patient object at 0x0000024CBA15DBD0>, <domain.entities.Patient object at 0x0000024CBA15CA10>), <domain.entities.Patient object at 0x0000024CBA15DFD0>, <domain.entities.Patient object at 0x0000024CBA15DBD0>, <domain.entities.Patient object at 0x0000024CBA15CA10>]


In [18]:
cp_check = []
a = []
for patient_combo in cp:
    if type(patient_combo) == tuple:
        for patient in patient_combo:
            patient_schedule = patient.schedule.get_numeric_template()
            a.append(patient_schedule)

        f = a[0]
        chk = True
        for item in a:
            if f != item:
                chk = False
                break
        if chk == True:
            cp_check.append((patient_combo, day))
        else:
            continue
    else:
        cp_check.append(patient_combo)
print(cp_check)

[<domain.entities.Patient object at 0x0000024CBA15DFD0>, <domain.entities.Patient object at 0x0000024CBA15DBD0>, <domain.entities.Patient object at 0x0000024CBA15CA10>]


In [7]:
# Desicion variables for each combination of therapist and patient
x = LpVariable.dicts("x", (ct_check, cp_check), lowBound=0, upBound=1, cat=plp.LpInteger)

In [ ]:
# Create the 'prob' variable to contain the problem data
prob = LpProblem("Therapist Patient Assignment", LpMaximize)